In [2]:
ENV["DYLD_LIBRARY_PATH"] ="/Users/mgrecu/CM1/cm1r21.1/run/:/Users/mgrecu/miniconda3/lib/"
ENV["LD_LIBRARY_PATH"] ="/Users/mgrecu/CM1/cm1r21.1/run/:/Users/mgrecu/miniconda3/lib/"
ENV["GFORTRAN_STDOUT_UNIT"]="166"

ccall((:cm1_init_, "cm1lib"), Cvoid, ())
dims=Int32.([100,120,30,1])
println(dims)
ccall((:get_dims_,"cm1lib"),Cvoid,(Ptr{Int32},),dims)
println(dims)
nx,ny,nz=dims[1],dims[2],dims[3]
numq=dims[4]
println(nx,ny,nz)
u=Array{Float32}(undef,nx+1,ny,nz);
v=Array{Float32}(undef,nx,ny+1,nz);
w=Array{Float32}(undef,nx,ny,nz+1);
ccall((:get_uvw_,"cm1lib"),Cvoid,(Ptr{Float32},Ptr{Float32},Ptr{Float32},Ptr{Int32},Ptr{Int32},Ptr{Int32},),u,v,w,dims[1:1],dims[2:2],dims[3:3])
prs=Array{Float32}(undef,nx,ny,nz);
ccall((:get_prs_,"cm1lib"),Cvoid,(Ptr{Float32},Ptr{Int32},Ptr{Int32},Ptr{Int32},),prs,dims[1:1],dims[2:2],dims[3:3])




 |---------------------------------------------------------------|
 |                                                               |
 |          You are using CM1, Release 21.1 (cm1r21.1)           |
 |                                                               |
 |---------------------------------------------------------------|

 Inside PARAM

   Domain dimensions: 
 nx    =         150
 ny    =         150
 nz    =          80

   sub-domain dimensions: 
 ni    =         150
 nj    =         150
 nk    =          80



  cm1setup =  1 : Large-Eddy Simulation (LES) 



  psolver =  3 : compressible, vertically implicit p-grad, with time-splitting



  Using these physical parameterizations: 

    microphysics             :  Morrison double moment                                      
    LES subgrid turbulence   :  Deardorff TKE scheme                                        
    planetary boundary layer :  no PBL model                                                
    surface l

In [3]:
print(dims)

Int32[150, 150, 80, 10]

In [4]:
using PyCall
np=pyimport("numpy")
np.random.seed(1969)
noise=np.random.randn(nx,ny,nz)
noise=noise.*0.5
ndimage=pyimport("scipy.ndimage")

noise=ndimage.gaussian_filter(noise,3)
fact=np.exp(noise);
alpha=1.5
for i in 1:nx
    for j in 1:ny
        x=(i-nx/2)*1.5
        y=(j-ny/2)*1.5
        w=exp(-alpha*((x/100)^2+(y/100)^2));
        fact[i,j,:]=1.0.+(fact[i,j,:].-1).*w
    end
end

In [5]:
q=Array{Float32}(undef,nx,ny,nz,numq);
ccall((:get_q_,"cm1lib"),Cvoid,(Ptr{Float32},Ptr{Int32},Ptr{Int32},Ptr{Int32},Ptr{Int32}),q,dims[1:1],dims[2:2],dims[3:3],dims[4:4])
q[:,:,:,1]=q[:,:,:,1].*fact;


         150         150          80


In [6]:
ccall((:set_q_,"cm1lib"),Cvoid,(Ptr{Float32},Ptr{Int32},Ptr{Int32},Ptr{Int32},Ptr{Int32}),q,dims[1:1],dims[2:2],dims[3:3],dims[4:4])


         150         150          80


In [ ]:
nsteps=Int32.([3*3600/4+1])
ccall((:cm1_time_step_,"cm1lib"),Cvoid,(Ptr{Int32},),nsteps)

In [8]:
mtime=ccall((:get_time_,"cm1lib"),Float32,())

1596.0f0

In [9]:
println(7200/4)

1800.0
